In [75]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [76]:
files = glob('url/*.json')
len(files)

65389

In [77]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        urls.extend(json.load(fopen))

100%|█████████████████████████████████| 65389/65389 [00:00<00:00, 106988.07it/s]


In [122]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    try:
        title = soup.find('h2', {'class': 'main-title'}).text
    except:
        title = None
    try:
        p = [p_.text for p_ in (soup.find('article').find_all('p'))]
    except:
        p = None
    
    data = {
        'url': url,
        'title': title,
        'p': p,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [123]:
# !rm -rf page
!mkdir page

mkdir: cannot create directory ‘page’: File exists


In [124]:
urls = sorted(list(set(urls)))
len(urls)

281766

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    aranged = np.arange(i, i + max_worker)
    urls_ = [(urls[a], os.path.join('page', f'{a}.json')) for a in aranged]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

    for future in as_completed(futures):
        future.result()

 72%|█████████████████████████          | 20171/28177 [1:53:56<55:35,  2.40it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████████████████████████████▊| 28064/28177 [2:57:50<00:46,  2.43it/s]

In [117]:
crawl(urls[10000], 'page/1.json')

In [100]:
r = requests.get(urls[10000])

In [101]:
soup = BeautifulSoup(r.content, "lxml")

In [112]:
soup.find('article').find_all('p')[0].text

'MISTERI kehilangan seorang warga emas berusia 87 tahun sejak dua hari lepas terungkai selepas orang awam menemui mayatnya terapung dalam longkang sawah di Parit 8, Chikus, di Teluk Intan petang semalam.'

In [113]:
title = soup.find('h2', {'class': 'main-title'}).text
p = [p_.text for p_ in (soup.find('article').find_all('p'))]
title, p

('Mayat warga emas ditemui dalam longkang sawah',
 ['MISTERI kehilangan seorang warga emas berusia 87 tahun sejak dua hari lepas terungkai selepas orang awam menemui mayatnya terapung dalam longkang sawah di Parit 8, Chikus, di Teluk Intan petang semalam.',
  'Ketua Polis Daerah Hilir Perak, ACP Mohd Marzukhi Mohd Mokhtar berkata Hamidah Hussin dari Kampung Sungai Tukang Sidin, Langkap, pada mulanya dilaporkan hilang selepas keluar dari rumahnya di Parit 8 sekitar pukul 7.00 malam Sabtu lalu.'])